# ---
## Module Import
---

In [1]:
import pandas as pd
import os
import gensim
import numpy as np
import time
import pickle
import warnings
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from gensim.models.fasttext import FastText as FT_gensim
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv1D, GlobalMaxPool1D, SimpleRNN
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

Using TensorFlow backend.


---
## HTTP_KNOWN DB
---

In [3]:
import pandas as pd
import pymysql

conn = pymysql.connect(host ='192.168.6.89' , port = 3306,
                       user = 'atom', password = 'atom', db = '5G_Probe_App_Discovery', charset = 'utf8', 
                      cursorclass = pymysql.cursors.DictCursor)

# 커서 가져오기 
curs = conn.cursor()

sql = "SELECT * FROM T_AD_XDR_HTTP_KNOWN"
curs.execute(sql)
http = curs.fetchall()
http = pd.DataFrame(http)

In [4]:
print('http_appid 개수:' + str(len(list(http.groupby('app_id').count().index))))

http_appid 개수:554


---
#### 데이터 만들때는 app_id 개수가 3개이하인건 제외하고 생각
#### 이상치 알고리즘에서 4개이상인것만 신경쓰도록 해야한다. 4개이하인 app_id 개수 : 110개
---

In [5]:
http['new_label'] = http['app_id']

In [6]:
appid_4 = list(http.groupby('app_id').count().seq.sort_values()[110: ].index)

In [7]:
for i in range(len(appid_4)):
    if i < len(appid_4)-1:
        http.loc[list(http[http.app_id == appid_4[i]].iloc[-1 : ].index)[0], 'new_label'] = appid_4[i+1]
        http.loc[list(http[http.app_id == appid_4[i]].iloc[ : -1].index)[:], 'new_label'] = appid_4[i]
    else : 
        http.loc[list(http[http.app_id == appid_4[i]].iloc[-1 : ].index)[0], 'new_label'] = appid_4[0] 
        http.loc[list(http[http.app_id == appid_4[i]].iloc[ : -1].index)[:], 'new_label'] = appid_4[i]  

---
## TRANSPORT_KNOWN DB
---

In [8]:
curs = conn.cursor()

sql = "SELECT * FROM T_AD_XDR_TR_KNOWN"
curs.execute(sql)
transport = curs.fetchall()
transport = pd.DataFrame(transport)

In [9]:
print('transport_appid 개수:' + str(len(list(transport.groupby('app_id').count().index))))

transport_appid 개수:563


---
#### 데이터 만들때는 app_id 개수가 3개이하인건 제외하고 생각
#### 이상치 알고리즘에서 4개이상인것만 신경쓰도록 해야한다
---

In [10]:
transport['new_label'] = transport['app_id']

In [11]:
appid_4 = list(transport.groupby('app_id').count().seq.sort_values()[110: ].index)

In [12]:
for i in range(len(appid_4)):
    if i < len(appid_4)-1:
        transport.loc[list(transport[transport.app_id == appid_4[i]].iloc[-1 : ].index)[0], 'new_label'] = appid_4[i+1]
        transport.loc[list(transport[transport.app_id == appid_4[i]].iloc[ : -1].index)[:], 'new_label'] = appid_4[i]
    else : 
        transport.loc[list(transport[transport.app_id == appid_4[i]].iloc[-1 : ].index)[0], 'new_label'] = appid_4[0] 
        transport.loc[list(transport[transport.app_id == appid_4[i]].iloc[ : -1].index)[:], 'new_label'] = appid_4[i]  

---
## DNS Vector화
---

In [13]:
from DNS_Encoding.data_nodns_del import data_nodns_del
from DNS_Encoding.make_fasttext_model_vector2 import make_fasttext_model_vector2
from DNS_Encoding.making_training_bydns import making_training_bydns
http_1 = data_nodns_del(http)
http_vector, http_modelinit, max_length_http = make_fasttext_model_vector2(http_1)
httpdns_training = making_training_bydns(http, http_vector)

In [14]:
transport_1 = data_nodns_del(transport)
transport_vector, transport_modelinit, max_length_transport = make_fasttext_model_vector2(transport_1)
transportdns_training = making_training_bydns(transport, transport_vector)

---
## IP Vector화
---

In [15]:
from IP_Encoding.ipencoding_65536_delH import ipencoding_65536_delH

http_ip65536_delH = ipencoding_65536_delH(http)
transport_ip65536_delH = ipencoding_65536_delH(transport)

/data01/CSB/CSB_Jupyter/PROBE/DEMO/IP_Encoding/ipencoding_65536_delH.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ip['app_ip_v4'] = data_ip.loc[:, 'app_ip_v4'].apply(hex).str.replace('0x', '').str.zfill(8)
/data01/CSB/CSB_Jupyter/PROBE/DEMO/IP_Encoding/ipencoding_65536_delH.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ip['app_ip_v4'] = "0:0:0:0:0:ffff:" + data_ip.loc[:, 'app_ip_v4'].str.slice(0, 4)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/pandas/core/indexing.py:

---
## ETC Features Vecotr화 : n+1 해결해야함
---

In [16]:
from Etc_features.age_onehot import age_onehot
from Etc_features.gender_onehot import gender_onehot
from Etc_features.port_onehot import port_onehot
from Etc_features.protocol_onehot import protocol_onehot
from Etc_features.time_hour_onehot import time_hour_onehot
from Etc_features.train_delta_dn_link_data_size import train_delta_dn_link_data_size
from Etc_features.train_delta_up_link_data_size import train_delta_up_link_data_size
from Etc_features.train_lastdnstimetolive import train_lastdnstimetolive
from Etc_features.train_synackmsssize import train_synackmsssize

http_age = age_onehot(http)
http_gender = gender_onehot(http)
http_port = port_onehot(http)
http_protocol = protocol_onehot(http)
http_timehour = time_hour_onehot(http)
http_lastdnstimetolive = train_lastdnstimetolive(http)
http_delta_dn_link_data_size = train_delta_dn_link_data_size(http)
http_delta_up_link_data_size = train_delta_up_link_data_size(http)
http_synackmsssize = train_synackmsssize(http)

http_training_ip65536_delH = pd.concat([http_ip65536_delH, httpdns_training, http_age, http_gender, http_port, http_protocol, http_timehour, http_lastdnstimetolive,http_delta_dn_link_data_size,http_delta_up_link_data_size,http_synackmsssize], axis = 1)


/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/data01/CSB/CSB_Jupyter/PROBE/DEMO/Etc_features/age_onehot.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [17]:
transport_age = age_onehot(transport)
transport_gender = gender_onehot(transport)
transport_port = port_onehot(transport)
transport_protocol = protocol_onehot(transport)
transport_timehour = time_hour_onehot(transport)
transport_lastdnstimetolive = train_lastdnstimetolive(transport)
transport_delta_dn_link_data_size = train_delta_dn_link_data_size(transport)
transport_delta_up_link_data_size = train_delta_up_link_data_size(transport)
transport_synackmsssize = train_synackmsssize(transport)

transport_training_ip65536_delH = pd.concat([transport_ip65536_delH, transportdns_training, transport_age, transport_gender, transport_port, transport_protocol, transport_timehour, transport_lastdnstimetolive,transport_delta_dn_link_data_size,transport_delta_up_link_data_size,transport_synackmsssize], axis = 1)

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/data01/CSB/CSB_Jupyter/PROBE/DEMO/Etc_features/age_onehot.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

---
## HTTP
---

In [18]:
app_id = list(http.groupby('app_id').count().index)
http_training_ip65536_delH = pd.concat([http_training_ip65536_delH , http[['app_id']]], axis=1)
http_training_ip65536_delH = pd.concat([http_training_ip65536_delH , http[['new_label']]], axis=1)

---
#### isolation forest 1, -1
---

In [19]:
a = pd.DataFrame(columns = http_training_ip65536_delH.columns)


for i in range(len(app_id)):
    if len(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]])<4:
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

        
###### 2는 3개이하짜리들 
    
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.10, random_state=35, verbose=0)
        y_pred = clf.fit_predict(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1)) #1,-1로 나온다.
        
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning

In [20]:
x = sum(a[a.app_id != a.new_label].anomaly_score == -1)
y = sum(a[a.app_id != a.new_label].anomaly_score == 1)
print('이상치 중에서 실제로 이상치로 모델이 판별한 비율 : ' + str(x/(x+y)*100))

이상치 중에서 실제로 이상치로 모델이 판별한 비율 : 21.17117117117117


In [21]:
z = sum(a[a.app_id == a.new_label].anomaly_score == -1)
w = sum(a[a.app_id == a.new_label].anomaly_score == 1)
print('정상 중에서 이상치로 잘못 판별한 비율 : ' + str(z/(z+w)*100))

정상 중에서 이상치로 잘못 판별한 비율 : 10.056058775412186


---
#### isolation forest score 값
---

In [22]:
a = pd.DataFrame(columns = http_training_ip65536_delH.columns)

for i in range(len(app_id)):
    if len(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]])<4:
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

            
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.10, random_state=35, verbose=0)
        clf.fit(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1))
        y_pred = clf.score_samples(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1)) #1,-1로 나온다.
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/h

In [23]:
print('전체 데이터 개수 : '+str(len(a)))
print('모델이 이상치로 판별한 데이터 개수 : ' + str(len(a[a.anomaly_score<-0.5])))
print('데이터 실제 이상치 개수 : ' + str(sum(a.app_id != a.new_label)))
print('이상치로 판별한 데이터 중 실제 이상치 개수 : '+ str(sum(a[a.anomaly_score<-0.5].app_id != a[a.anomaly_score<-0.5].new_label)))

전체 데이터 개수 : 133915
모델이 이상치로 판별한 데이터 개수 : 9272
데이터 실제 이상치 개수 : 444
이상치로 판별한 데이터 중 실제 이상치 개수 : 160


---
#### 권대리님 모델에 태울 것
---

In [35]:
http.loc[list(a[a.anomaly_score <-0.5].index)].to_csv('modelinputhttp.csv')

---
## Transport
---

In [24]:
app_id = list(transport.groupby('app_id').count().index)
transport_training_ip65536_delH = pd.concat([transport_training_ip65536_delH , transport[['app_id']]], axis=1)
transport_training_ip65536_delH = pd.concat([transport_training_ip65536_delH , transport[['new_label']]], axis=1)

---
#### isolation forest 1, -1
---

In [25]:
b = pd.DataFrame(columns = transport_training_ip65536_delH.columns)


for i in range(len(app_id)):
    if len(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]])<4:
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        b = pd.concat([b,xx])

        
###### 2는 3개이하짜리들 
    
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.10, random_state=35, verbose=0)
        y_pred = clf.fit_predict(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1)) #1,-1로 나온다.
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        b = pd.concat([b,xx])
        
        
###### -1이 이상치, 1이 정상.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning

In [26]:
x = sum(b[b.app_id != b.new_label].anomaly_score == -1)
y = sum(b[b.app_id != b.new_label].anomaly_score == 1)
print('이상치 중에서 실제로 이상치로 모델이 판별한 비율 : ' + str(x/(x+y)*100))

이상치 중에서 실제로 이상치로 모델이 판별한 비율 : 15.894039735099339


In [27]:
z = sum(b[b.app_id == b.new_label].anomaly_score == -1)
w = sum(b[b.app_id == b.new_label].anomaly_score == 1)
print('정상 중에서 이상치로 잘못 판별한 비율 : ' + str(z/(z+w)*100))

정상 중에서 이상치로 잘못 판별한 비율 : 10.015263307130319


---
#### isolation forest score 값
---

In [29]:
b = pd.DataFrame(columns = transport_training_ip65536_delH.columns)

for i in range(len(app_id)):
    if len(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]])<4:
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        b = pd.concat([a,xx])

            
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.10, random_state=35, verbose=0)
        clf.fit(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1))
        y_pred = clf.score_samples(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1)) #1,-1로 나온다.
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        b = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/h

In [31]:
print('전체 데이터 개수 : '+str(len(b)))
print('모델이 이상치로 판별한 데이터 개수 : ' + str(len(b[b.anomaly_score<-0.5])))
print('데이터 실제 이상치 개수 : ' + str(sum(b.app_id != b.new_label)))
print('이상치로 판별한 데이터 중 실제 이상치 개수 : '+ str(sum(b[b.anomaly_score<-0.5].app_id != b[b.anomaly_score<-0.5].new_label)))

전체 데이터 개수 : 134502
모델이 이상치로 판별한 데이터 개수 : 9313
데이터 실제 이상치 개수 : 445
이상치로 판별한 데이터 중 실제 이상치 개수 : 160


---
#### 권 대리님 모델에 태울 것
---

In [43]:
transport.loc[list((b.anomaly_score<-0.5).index)].to_csv('modelinputtransport.csv')